In [16]:
import sys
from pathlib import Path
import numpy as np
from PIL import Image
from dotenv import load_dotenv
import pandas as pd
from inference_sdk import InferenceHTTPClient
from PIL import Image, ImageEnhance
from IPython.display import clear_output
import os
import numpy as np

# 🔎 Busquem la carpeta arrel del projecte: "potato-dry-matter-optics-ml"
current_path = Path().resolve()
TARGET_ROOT_NAME = "potato-dry-matter-optics-ml"

while current_path.name != TARGET_ROOT_NAME and current_path.parent != current_path:
    current_path = current_path.parent

if current_path.name != TARGET_ROOT_NAME:
    raise RuntimeError(
        f"No s'ha trobat la carpeta '{TARGET_ROOT_NAME}' pujant des del directori actual: {Path().resolve()}"
    )

project_root = current_path
sys.path.append(str(project_root))

# Cargar variables de entorno (ROBOFLOW_API_KEY)
load_dotenv()

# Import del mètode a testejar
from src.raw_image_treatment import potato_defect_classification, potato_pixels_rgb_img

print(f"✅ Project root: {project_root}")


✅ Project root: C:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml


In [3]:
# Carpeta d'imatges
img_dir = project_root / "data/input/raw/raw_images/test_1"

# Llistat d'imatges del lot 1
paths = []
paths += [img_dir / f"p1_{i}.png" for i in range(1, 25)]
paths += [img_dir / f"p2_{j}.png" for j in range(1, 12)]

In [6]:
# Client Roboflow (per extreure la confidence de la millor predicció)
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"  # posa el mateix model que uses a src
api_key = os.environ["ROBOFLOW_API_KEY"]
client = InferenceHTTPClient(api_url="https://serverless.roboflow.com", api_key=api_key)

rows = []
thr = 0.0
total = len(paths)

for k, p in enumerate(paths, start=1):
    clear_output(wait=True)
    print(f"⏳ Processant patata {k}/{total}: {p.name}")

    if not p.exists():
        rows.append({
            "filename": p.name,
            "path": str(p),
            "defect": np.nan,
            "confidence": np.nan,
            "n_predictions": 0,
            "status": "missing_file",
        })
        continue

    # 1) Defecte (segons la funció del src)
    defect, _vis = potato_defect_classification(p, confidence_threshold=thr)

    # 2) Confidence de la millor predicció (inferencia directa)
    result = client.infer(str(p), model_id=ROBOFLOW_MODEL_ID)
    preds = result.get("predictions", []) if isinstance(result, dict) else []

    if preds:
        best = max(preds, key=lambda d: float(d.get("confidence", 0.0)))
        conf = float(best.get("confidence", 0.0))
        n_preds = len(preds)
        status = "ok"
    else:
        conf = np.nan
        n_preds = 0
        status = "no_predictions"

    rows.append({
        "filename": p.name,
        "defect": defect,
        "confidence": conf,
        "n_predictions": n_preds,
        "status": status,
    })

clear_output(wait=True)
print(f"✅ Completat: {total}/{total} imatges processades.")

df = pd.DataFrame(rows)

# Ordenar: primer p1_*, després p2_*
def _sort_key(name: str):
    stem = Path(name).stem  # p1_10
    part, idx = stem.split("_")
    lot = int(part[1:])     # 1 o 2
    return (lot, int(idx))

df = df.sort_values("filename", key=lambda s: s.map(_sort_key)).reset_index(drop=True)

df


✅ Completat: 35/35 imatges processades.


,filename,defect,confidence,n_predictions,status
0,p1_1.png,Damaged potato,0.540028,1,ok
1,p1_2.png,Potato,NaN,0,no_predictions
2,p1_3.png,Damaged potato,0.664749,1,ok
3,p1_4.png,Damaged potato,0.528479,2,ok
4,p1_5.png,Damaged potato,0.690631,1,ok
5,p1_6.png,Diseased-fungal potato,0.551619,2,ok
6,p1_7.png,Damaged potato,0.641539,1,ok
7,p1_8.png,Potato,0.600510,2,ok
8,p1_9.png,Damaged potato,0.483370,2,ok
9,p1_10.png,Damaged potato,NaN,0,no_predictions


In [7]:
# Client Roboflow
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"
api_key = os.environ["ROBOFLOW_API_KEY"]
client = InferenceHTTPClient(api_url="https://serverless.roboflow.com", api_key=api_key)

rows_bright = []
thr = 0.0
total = len(paths)

for k, p in enumerate(paths, start=1):
    clear_output(wait=True)
    print(f"💡 (Brightness x1.5) Processant patata {k}/{total}: {p.name}")

    if not p.exists():
        rows_bright.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "missing_file",
        })
        continue

    # Carregar imatge i augmentar brightness
    pil = Image.open(p).convert("RGB")
    pil_bright = ImageEnhance.Brightness(pil).enhance(1.5)

    # Inferència directa (per confidence)
    result = client.infer(pil_bright, model_id=ROBOFLOW_MODEL_ID)
    preds = result.get("predictions", [])

    if preds:
        best = max(preds, key=lambda d: float(d.get("confidence", 0.0)))
        defect = best.get("class")
        conf = float(best.get("confidence", 0.0))
        status = "ok"
    else:
        defect = np.nan
        conf = np.nan
        status = "no_predictions"

    rows_bright.append({
        "filename": p.name,
        "defect": defect,
        "confidence": conf,
        "status": status,
    })

clear_output(wait=True)
print("✅ Brightness x1.5 completat")

df_bright = pd.DataFrame(rows_bright)
df_bright


✅ Brightness x1.5 completat


,filename,defect,confidence,status
0,p1_1.png,Potato,0.668413,ok
1,p1_2.png,Diseased-fungal potato,0.797845,ok
2,p1_3.png,Damaged potato,0.536085,ok
3,p1_4.png,Diseased-fungal potato,0.699943,ok
4,p1_5.png,Damaged potato,0.721254,ok
5,p1_6.png,Diseased-fungal potato,0.613298,ok
6,p1_7.png,Damaged potato,0.431860,ok
7,p1_8.png,Damaged potato,0.453655,ok
8,p1_9.png,Diseased-fungal potato,0.662893,ok
9,p1_10.png,Damaged potato,0.494689,ok


In [4]:
# Roboflow client
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"  # mismo modelo que defect classification
api_key = os.environ["ROBOFLOW_API_KEY"]
client = InferenceHTTPClient(api_url="https://serverless.roboflow.com", api_key=api_key)

rows_cropped = []
thr = 0.0
total = len(paths)

for k, p in enumerate(paths, start=1):
    clear_output(wait=True)
    print(f"✂️ (Cropped→Defect) Processant patata {k}/{total}: {p.name}")

    if not p.exists():
        rows_cropped.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "missing_file",
        })
        continue

    # 1) Obtenir cropped_img (segmentació)
    cropped_img, _vis = potato_pixels_rgb_img(p, margin=50)

    if cropped_img is None:
        rows_cropped.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "no_mask",
        })
        continue

    # 2) Predicció de defecte sobre el cropped (amb la funció del src)
    defect, _vis_def = potato_defect_classification(cropped_img, confidence_threshold=thr)

    # 3) Confidence (inferencia directa sobre el cropped)
    result = client.infer(cropped_img, model_id=ROBOFLOW_MODEL_ID)
    preds = result.get("predictions", []) if isinstance(result, dict) else []

    if preds:
        best = max(preds, key=lambda d: float(d.get("confidence", 0.0)))
        conf = float(best.get("confidence", 0.0))
        status = "ok"
    else:
        conf = np.nan
        status = "no_predictions"

    rows_cropped.append({
        "filename": p.name,
        "defect": defect,
        "confidence": conf,
        "status": status,
    })

clear_output(wait=True)
print("✅ Cropped→Defect completat")

df_cropped = pd.DataFrame(rows_cropped)
df_cropped

✅ Cropped→Defect completat


,filename,defect,confidence,status
0,p1_1.png,NaN,NaN,no_predictions
1,p1_2.png,Potato,0.794069,ok
2,p1_3.png,NaN,NaN,no_predictions
3,p1_4.png,NaN,NaN,no_predictions
4,p1_5.png,Damaged potato,0.800655,ok
5,p1_6.png,Potato,0.579518,ok
6,p1_7.png,Sprouted potato,0.490660,ok
7,p1_8.png,Potato,0.632041,ok
8,p1_9.png,Potato,0.533334,ok
9,p1_10.png,Potato,0.713344,ok


In [9]:
# Roboflow client
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"
api_key = os.environ["ROBOFLOW_API_KEY"]
client = InferenceHTTPClient(api_url="https://serverless.roboflow.com", api_key=api_key)

rows_cropped_m50 = []
thr = 0.0
total = len(paths)

for k, p in enumerate(paths, start=1):
    clear_output(wait=True)
    print(f"✂️ (Cropped m=50 → Defect) Processant patata {k}/{total}: {p.name}")

    if not p.exists():
        rows_cropped_m50.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "missing_file",
        })
        continue

    # 1) Obtenir cropped_img (segmentació) amb margin=50
    cropped_img, _vis = potato_pixels_rgb_img(p, margin=50)

    if cropped_img is None:
        rows_cropped_m50.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "no_mask",
        })
        continue

    # 2) Predicció de defecte sobre el cropped
    defect, _vis_def = potato_defect_classification(cropped_img, confidence_threshold=thr)

    # 3) Confidence (inferencia directa sobre el cropped)
    result = client.infer(cropped_img, model_id=ROBOFLOW_MODEL_ID)
    preds = result.get("predictions", []) if isinstance(result, dict) else []

    if preds:
        best = max(preds, key=lambda d: float(d.get("confidence", 0.0)))
        conf = float(best.get("confidence", 0.0))
        status = "ok"
    else:
        conf = np.nan
        status = "no_predictions"

    rows_cropped_m50.append({
        "filename": p.name,
        "defect": defect,
        "confidence": conf,
        "status": status,
    })

clear_output(wait=True)
print("✅ Cropped(margin=50) → Defect completat")

df_cropped_m50 = pd.DataFrame(rows_cropped_m50)
df_cropped_m50

✅ Cropped(margin=50) → Defect completat


,filename,defect,confidence,status
0,p1_1.png,NaN,NaN,no_predictions
1,p1_2.png,Potato,0.820365,ok
2,p1_3.png,Potato,0.570155,ok
3,p1_4.png,NaN,NaN,no_predictions
4,p1_5.png,Damaged potato,0.827644,ok
5,p1_6.png,Potato,0.760823,ok
6,p1_7.png,NaN,NaN,no_predictions
7,p1_8.png,Potato,0.514052,ok
8,p1_9.png,Potato,0.740610,ok
9,p1_10.png,Potato,0.571669,ok


In [11]:
# Roboflow client
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"
api_key = os.environ["ROBOFLOW_API_KEY"]
client = InferenceHTTPClient(api_url="https://serverless.roboflow.com", api_key=api_key)

rows_cropped_m50_bright = []
thr = 0.0
total = len(paths)

for k, p in enumerate(paths, start=1):
    clear_output(wait=True)
    print(f"✂️💡 (Cropped m=50 + Brightness x1.5) {k}/{total}: {p.name}")

    if not p.exists():
        rows_cropped_m50_bright.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "missing_file",
        })
        continue

    # 1) Cropped amb margin=50
    cropped_img, _vis = potato_pixels_rgb_img(p, margin=50)

    if cropped_img is None:
        rows_cropped_m50_bright.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "no_mask",
        })
        continue

    # 2) Brightness x1.5 sobre el cropped
    cropped_bright = ImageEnhance.Brightness(cropped_img).enhance(1.5)

    # 3) Predicció defecte
    defect, _vis_def = potato_defect_classification(cropped_bright, confidence_threshold=thr)

    # 4) Confidence directa
    result = client.infer(cropped_bright, model_id=ROBOFLOW_MODEL_ID)
    preds = result.get("predictions", []) if isinstance(result, dict) else []

    if preds:
        best = max(preds, key=lambda d: float(d.get("confidence", 0.0)))
        conf = float(best.get("confidence", 0.0))
        status = "ok"
    else:
        conf = np.nan
        status = "no_predictions"

    rows_cropped_m50_bright.append({
        "filename": p.name,
        "defect": defect,
        "confidence": conf,
        "status": status,
    })

clear_output(wait=True)
print("✅ Cropped(m=50) + Brightness x1.5 → Defect completat")

df_cropped_m50_bright = pd.DataFrame(rows_cropped_m50_bright)
df_cropped_m50_bright

✅ Cropped(m=50) + Brightness x1.5 → Defect completat


,filename,defect,confidence,status
0,p1_1.png,Potato,0.846212,ok
1,p1_2.png,Potato,0.897467,ok
2,p1_3.png,Potato,0.614110,ok
3,p1_4.png,Defected potato,0.408177,ok
4,p1_5.png,Damaged potato,0.656056,ok
5,p1_6.png,Potato,0.869114,ok
6,p1_7.png,Diseased-fungal potato,0.400120,ok
7,p1_8.png,Potato,0.508336,ok
8,p1_9.png,Potato,0.769224,ok
9,p1_10.png,Potato,0.736523,ok


In [13]:
# Roboflow client
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"
api_key = os.environ["ROBOFLOW_API_KEY"]
client = InferenceHTTPClient(api_url="https://serverless.roboflow.com", api_key=api_key)

rows_bright2 = []
thr = 0.0
total = len(paths)

for k, p in enumerate(paths, start=1):
    clear_output(wait=True)
    print(f"💡 (Brightness x2.0) Processant patata {k}/{total}: {p.name}")

    if not p.exists():
        rows_bright2.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "missing_file",
        })
        continue

    # 1) Carregar imatge i aplicar brightness x2.0
    pil = Image.open(p).convert("RGB")
    pil_bright2 = ImageEnhance.Brightness(pil).enhance(2.0)

    # 2) Inferència directa
    result = client.infer(pil_bright2, model_id=ROBOFLOW_MODEL_ID)
    preds = result.get("predictions", [])

    if preds:
        best = max(preds, key=lambda d: float(d.get("confidence", 0.0)))
        defect = best.get("class")
        conf = float(best.get("confidence", 0.0))
        status = "ok"
    else:
        defect = np.nan
        conf = np.nan
        status = "no_predictions"

    rows_bright2.append({
        "filename": p.name,
        "defect": defect,
        "confidence": conf,
        "status": status,
    })

clear_output(wait=True)
print("✅ Brightness x2.0 completat")

df_bright2 = pd.DataFrame(rows_bright2)
df_bright2

✅ Brightness x2.0 completat


,filename,defect,confidence,status
0,p1_1.png,Potato,0.890788,ok
1,p1_2.png,Potato,0.767066,ok
2,p1_3.png,Diseased-fungal potato,0.493515,ok
3,p1_4.png,Potato,0.656483,ok
4,p1_5.png,Defected potato,0.747180,ok
5,p1_6.png,Potato,0.840185,ok
6,p1_7.png,Damaged potato,0.806620,ok
7,p1_8.png,Potato,0.725757,ok
8,p1_9.png,Potato,0.820156,ok
9,p1_10.png,Damaged potato,0.561101,ok


In [15]:
# Roboflow client
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"
api_key = os.environ["ROBOFLOW_API_KEY"]
client = InferenceHTTPClient(api_url="https://serverless.roboflow.com", api_key=api_key)

rows_bright25 = []
thr = 0.0
total = len(paths)

for k, p in enumerate(paths, start=1):
    clear_output(wait=True)
    print(f"💡 (Brightness x2.5) Processant patata {k}/{total}: {p.name}")

    if not p.exists():
        rows_bright25.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "missing_file",
        })
        continue

    pil = Image.open(p).convert("RGB")
    pil_bright25 = ImageEnhance.Brightness(pil).enhance(2.5)

    result = client.infer(pil_bright25, model_id=ROBOFLOW_MODEL_ID)
    preds = result.get("predictions", [])

    if preds:
        best = max(preds, key=lambda d: float(d.get("confidence", 0.0)))
        defect = best.get("class")
        conf = float(best.get("confidence", 0.0))
        status = "ok"
    else:
        defect = np.nan
        conf = np.nan
        status = "no_predictions"

    rows_bright25.append({
        "filename": p.name,
        "defect": defect,
        "confidence": conf,
        "status": status,
    })

clear_output(wait=True)
print("✅ Brightness x2.5 completat")

df_bright25 = pd.DataFrame(rows_bright25)
df_bright25

✅ Brightness x2.5 completat


,filename,defect,confidence,status
0,p1_1.png,Potato,0.900088,ok
1,p1_2.png,Potato,0.852937,ok
2,p1_3.png,Potato,0.894310,ok
3,p1_4.png,Potato,0.653453,ok
4,p1_5.png,Potato,0.866955,ok
5,p1_6.png,Potato,0.899824,ok
6,p1_7.png,Damaged potato,0.849948,ok
7,p1_8.png,Potato,0.891688,ok
8,p1_9.png,Potato,0.898736,ok
9,p1_10.png,Potato,0.482018,ok


In [17]:
# --- Gamma correction helper ---------------------------------------
def gamma_correction(pil_img, gamma=0.8):
    arr = np.asarray(pil_img).astype(np.float32) / 255.0
    arr = arr ** gamma
    arr = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
    return Image.fromarray(arr)

# Roboflow client
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"
api_key = os.environ["ROBOFLOW_API_KEY"]
client = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key=api_key
)

rows_bright2_gamma = []
thr = 0.0
total = len(paths)

for k, p in enumerate(paths, start=1):
    clear_output(wait=True)
    print(f"💡🌗 (Brightness x2 + Gamma 0.8) {k}/{total}: {p.name}")

    if not p.exists():
        rows_bright2_gamma.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "missing_file",
        })
        continue

    # 1) Load image
    pil = Image.open(p).convert("RGB")

    # 2) Brightness x2
    pil_b2 = ImageEnhance.Brightness(pil).enhance(2.0)

    # 3) Gamma correction
    pil_b2_gamma = gamma_correction(pil_b2, gamma=0.8)

    # 4) Inference
    result = client.infer(pil_b2_gamma, model_id=ROBOFLOW_MODEL_ID)
    preds = result.get("predictions", [])

    if preds:
        best = max(preds, key=lambda d: float(d.get("confidence", 0.0)))
        defect = best.get("class")
        conf = float(best.get("confidence", 0.0))
        status = "ok"
    else:
        defect = np.nan
        conf = np.nan
        status = "no_predictions"

    rows_bright2_gamma.append({
        "filename": p.name,
        "defect": defect,
        "confidence": conf,
        "status": status,
    })

clear_output(wait=True)
print("✅ Brightness x2 + Gamma (γ=0.8) completat")

df_bright2_gamma = pd.DataFrame(rows_bright2_gamma)
df_bright2_gamma

✅ Brightness x2 + Gamma (γ=0.8) completat


,filename,defect,confidence,status
0,p1_1.png,Potato,0.937543,ok
1,p1_2.png,Potato,0.870144,ok
2,p1_3.png,Potato,0.818213,ok
3,p1_4.png,Potato,0.866222,ok
4,p1_5.png,Potato,0.864748,ok
5,p1_6.png,Potato,0.919829,ok
6,p1_7.png,Damaged potato,0.788106,ok
7,p1_8.png,Potato,0.881254,ok
8,p1_9.png,Potato,0.899187,ok
9,p1_10.png,Potato,0.752340,ok


In [19]:
# --- Gamma correction helper ---------------------------------------
def gamma_correction(pil_img, gamma=0.8):
    arr = np.asarray(pil_img).astype(np.float32) / 255.0
    arr = arr ** gamma
    arr = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
    return Image.fromarray(arr)

# Roboflow client
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"
api_key = os.environ["ROBOFLOW_API_KEY"]
client = InferenceHTTPClient(
    api_url="https://serverless.roboflow.com",
    api_key=api_key
)

rows_bright25_gamma = []
thr = 0.0
total = len(paths)

for k, p in enumerate(paths, start=1):
    clear_output(wait=True)
    print(f"💡🌗 (Brightness x2.5 + Gamma 0.8) {k}/{total}: {p.name}")

    if not p.exists():
        rows_bright25_gamma.append({
            "filename": p.name,
            "defect": np.nan,
            "confidence": np.nan,
            "status": "missing_file",
        })
        continue

    # 1) Load image
    pil = Image.open(p).convert("RGB")

    # 2) Brightness x2.5
    pil_b25 = ImageEnhance.Brightness(pil).enhance(2.5)

    # 3) Gamma correction
    pil_b25_gamma = gamma_correction(pil_b25, gamma=0.8)

    # 4) Inference
    result = client.infer(pil_b25_gamma, model_id=ROBOFLOW_MODEL_ID)
    preds = result.get("predictions", [])

    if preds:
        best = max(preds, key=lambda d: float(d.get("confidence", 0.0)))
        defect = best.get("class")
        conf = float(best.get("confidence", 0.0))
        status = "ok"
    else:
        defect = np.nan
        conf = np.nan
        status = "no_predictions"

    rows_bright25_gamma.append({
        "filename": p.name,
        "defect": defect,
        "confidence": conf,
        "status": status,
    })

clear_output(wait=True)
print("✅ Brightness x2.5 + Gamma (γ=0.8) completat")

df_bright25_gamma = pd.DataFrame(rows_bright25_gamma)
df_bright25_gamma

✅ Brightness x2.5 + Gamma (γ=0.8) completat


,filename,defect,confidence,status
0,p1_1.png,Potato,0.932087,ok
1,p1_2.png,Potato,0.883967,ok
2,p1_3.png,Potato,0.928464,ok
3,p1_4.png,Potato,0.863369,ok
4,p1_5.png,Potato,0.905801,ok
5,p1_6.png,Potato,0.934917,ok
6,p1_7.png,Damaged potato,0.847408,ok
7,p1_8.png,Potato,0.927291,ok
8,p1_9.png,Potato,0.909183,ok
9,p1_10.png,Potato,0.804376,ok


In [20]:
def summarize(df, label):
    total = len(df)

    summary = (
        df.dropna(subset=["defect"])
        .groupby("defect")
        .agg(
            detections=("defect", "count"),
            mean_confidence=("confidence", "mean"),
        )
        .reset_index()
    )

    summary["% deteccio"] = 100 * summary["detections"] / total
    summary["mean_confidence_%"] = 100 * summary["mean_confidence"]
    summary["case"] = label

    return summary[["case", "defect", "% deteccio", "mean_confidence_%"]]


summary_all = pd.concat(
    [
        summarize(df, "original"),
        summarize(df_bright, "brightness_x1.5"),
        summarize(df_bright2, "brightness_x2.0"),
        summarize(df_bright25, "brightness_x2.5"),
        summarize(df_bright2_gamma, "brightness_x2_gamma"),
        summarize(df_bright25_gamma, "brightness_x2.5_gamma"),
        summarize(df_cropped, "cropped_then_defect_m0"),
        summarize(df_cropped_m50, "cropped_then_defect_m50"),
        summarize(df_cropped_m50_bright, "cropped_m50_brightness_x1.5"),
    ],
    ignore_index=True
)

summary_all.sort_values(
    ["case", "% deteccio"],
    ascending=[True, False]
)

,case,defect,% deteccio,mean_confidence_%
4,brightness_x1.5,Diseased-fungal potato,45.714286,68.868641
3,brightness_x1.5,Damaged potato,40.000000,55.524190
5,brightness_x1.5,Potato,14.285714,61.482910
9,brightness_x2.0,Potato,42.857143,76.690748
8,brightness_x2.0,Diseased-fungal potato,25.714286,64.923437
6,brightness_x2.0,Damaged potato,22.857143,64.308678
7,brightness_x2.0,Defected potato,5.714286,69.225389
10,brightness_x2.0,Sprouted potato,2.857143,54.056293
14,brightness_x2.5,Potato,60.000000,79.673934
11,brightness_x2.5,Damaged potato,28.571429,65.563475
